In [ ]:
#ภวัต นาคเจือทอง 6506022620109

In [58]:
!pip -q install pandas scikit-learn openpyxl streamlit pyngrok joblib

In [3]:
import os, re, json, random
import pandas as pd, joblib
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [128]:
# =========================================================
# 1. Facts, Templates, Helper, Build Dataset
# =========================================================

FACTS = {
    "year": "2025",
    "team_short": "Gazoo Racing",
    "team_full": "WRC Toyota Gazoo Racing",
    "drivers": ["Kalle Rovanperä", "Elfyn Evans", "Takamoto Katsuta", "Sébastien Ogier"],
    "team_principal": "Jari-Matti Latvala",
    "technical_director": "Tom Fowler",
    "car_name": "Toyota GR Yaris Rally1 Hybrid",
    "power_unit": "เครื่องยนต์เบนซิน 1.6 ลิตร เทอร์โบ ผสานกับระบบไฮบริด 100kW (134 แรงม้า) ซึ่งเป็นไปตามกติกา FIA Rally1",
    "base_chassis": "Jyväskylä, Finland (ศูนย์ใหญ่ของทีม)",
    "base_engine": "ญี่ปุ่น (ส่วนใหญ่) และยุโรป (บางส่วน)",

    # คะแนนล่าสุดผู้ผลิต
    "constructors_points_asof": "513 คะแนน — Toyota นำหน้า Hyundai และ M-Sport",
    "asof_event": "หลังจบการแข่งขัน Rally del Paraguay (สนามที่ 10)",
    "asof_date": "อัปเดตล่าสุด ณ วันที่ 14 กันยายน 2025",  # ตามข้อมูลเว็บทางการ
    "car_focus": "ทีมเน้นการปรับปรุงระบบแอร์โรไดนามิก ช่วงล่าง และการจัดการพลังงานไฮบริดเพื่อเพิ่มประสิทธิภาพในทุกสภาพถนน พร้อมปรับปรุงการตอบสนองของเครื่องยนต์",

    "standings": {
        "Elfyn Evans": {
            "pos": "อันดับ 1 คะแนนนักขับด้วย 198 คะแนน",
            "pts": "198",
            "nat": "GBR"
        },
        "Kalle Rovanperä": {
            "pos": "อันดับ 2 คะแนนนักขับด้วย 191 คะแนน",
            "pts": "191",
            "nat": "FIN"
        },
        "Sébastien Ogier": {
            "pos": "อันดับ 3 คะแนนนักขับด้วย 189 คะแนน",
            "pts": "189",
            "nat": "FRA"
        },
        "Ott Tänak": {
            "pos": "อันดับ 4 คะแนนนักขับด้วย 180 คะแนน",
            "pts": "180",
            "nat": "EST"
        },
        "Thierry Neuville": {
            "pos": "อันดับ 5 คะแนนนักขับด้วย 150 คะแนน",
            "pts": "150",
            "nat": "BEL"
        },
        "Takamoto Katsuta": {
            "pos": "อันดับ 6 คะแนนนักขับด้วย 88 คะแนน",
            "pts": "88",
            "nat": "JPN"
        },
        "manufacturers": {
            "Toyota Gazoo Racing WRT": "513",
            "Hyundai Shell Mobis World Rally Team": "413",
            "M-Sport Ford World Rally Team": "143"
        }
    },

    # ข้อมูลเพิ่มเติมที่เข้ากับ INTENT_TEMPLATES ใหม่
    "next_event": "Rally Chile Bio Bío (สนามที่ 11) วันที่ 11-14 กันยายน 2025",
    "team_strengths": "ความเสถียรของ {car_name}, นักขับชั้นนำ และการปรับแต่งรถแข่งให้เหมาะสมกับพื้นผิวที่หลากหลาย — Gazoo Racing ยืนหยัดในกลุ่มผู้นำทั้งในฝุ่น ดินแดง และกรวด",
    "team_rivals": ["Hyundai Shell Mobis World Rally Team", "M-Sport Ford World Rally Team"]
}


In [129]:
# ===== List → string สวย ๆ (รองรับไทย/อังกฤษ) =====
def _stringify(val, thai=False):
    if isinstance(val, (list, tuple)):
        if not val: return ""
        if len(val)==1: return str(val[0])
        conj = " และ " if thai else " and "
        return ", ".join(map(str, val[:-1])) + conj + str(val[-1])
    return str(val)

def fill_response(template: str, facts=FACTS):
    th = bool(re.search(r'[\u0E00-\u0E7F]', template))
    def repl(m):
        key = m.group(1)
        try:
            v = eval(key, {}, facts)  # รองรับ {drivers[0]} / facts["standings"]["George Russell"]["pts"]
        except Exception:
            v = facts.get(key, m.group(0))
        return _stringify(v, th)
    return re.sub(r"\{([^\}]+)\}", repl, template)

In [130]:
# ===== Intents & Templates =====
INTENT_TEMPLATES = {
    "greeting": {
        "questions": ["สวัสดี","หวัดดี","มีใครอยู่ไหม","hi","hello","hey", "ทักทาย", "โย่ว", "ทักครับ", "ไง", "ไงเพื่อน", "สวัสดีครับ", "สวัสดีค่ะ"],
        "responses": [
            "สวัสดีครับ/ค่ะ ยินดีต้อนรับสู่ WRC Toyota Gazoo Racing {year} 👋",
            "สวัสดีครับ ผมคือบอทผู้ช่วยด้านข้อมูล WRC Gazoo Racing พร้อมช่วยคุณเสมอ",
            "สวัสดีครับ/ค่ะ ถามเรื่องทีม WRC Gazoo Racing 2025 ได้เลย"
        ]
    },
    "goodbye": {
        "questions": ["ลาแล้ว","ไปก่อนนะ","ไว้เจอกัน","bye","goodbye","see you","แล้วเจอกัน", "บ๊ายบาย", "ไปละ", "ขอบคุณครับ", "ขอบคุณค่ะ", "ขอบคุณมาก", "พอแค่นี้", "จบการสนทนา"],
        "responses": [
            "โชคดีครับ แล้วเจอกันใหม่! 🏁",
            "ขอบคุณครับ/ค่ะ ไว้คุยกันใหม่นะ",
            "ขอให้สนุกกับการติดตาม WRC ครับ"
        ]
    },
    "capabilities": {
        "questions": ["บอทนี้ทำอะไรได้บ้าง","ขอบเขตบอทนี้","ช่วยเรื่อง {team_short} {year} ได้ไหม","what can you do", "บอทนี้ตอบคำถามอะไรได้บ้าง", "คุณตอบคำถามเกี่ยวกับอะไรได้บ้าง", "ความสามารถของคุณ", "ถามอะไรได้บ้าง", "ช่วยอะไรฉันได้บ้าง"],
        "responses": [
            "ผมสามารถให้ข้อมูลเกี่ยวกับทีม {team_full} ฤดูกาล {year} รวมถึงนักขับ, รถ, คะแนน, ทีมงาน และประวัติที่สำคัญของทีมได้ครับ"
        ]
    },
    "team_overview": {
        "questions": ["สรุปทีม {team_short} {year} ให้หน่อย","ภาพรวม {team_short} {year}","overview {team_short} {year}", "ข้อมูลเบื้องต้นเกี่ยวกับทีม", "แนะนำทีม {team_full} หน่อย", "ทีม Gazoo Racing คืออะไร", "ทีมนี้แข่งอะไร"],
        "responses": [
            "ทีม {team_full} เป็นทีมแข่งทางฝุ่นจาก Toyota ที่ลงแข่งรายการ WRC ใช้รถ {car_name} นักขับหลักปี {year} ได้แก่ {drivers[0]}, {drivers[1]} และ {drivers[2]}."
        ]
    },
    "team_goal": {
        "questions": ["ฤดูกาลนี้ทีมมีเป้าหมายอะไร","เป้าหมายของทีม", "เป้าหมายหลักของทีมคืออะไร", "ปีนี้ทีมตั้งเป้าไว้ยังไง", "ทีมอยากได้แชมป์อะไร", "ทีมมีเป้าหมายในฤดูกาลนี้ยังไงบ้าง"],
        "responses": [
            "เป้าหมายหลักของทีมคือการป้องกันแชมป์ Manufacturers และช่วยให้นักขับตัวเต็งอย่าง {drivers[0]} หรือ {drivers[1]} คว้าแชมป์โลก Drivers’ Championship ให้ได้"
        ]
    },
    "drivers_2025": {
        "questions": ["นักแข่ง", "คนขับ", "ใครขับ", "นักขับของ {team_short} {year} มีใครบ้าง","{team_short} {year} drivers","ไลน์อัพ {team_short} {year}", "รายชื่อนักขับปีนี้", "คนขับรถของทีมมีใครบ้าง", "มีนักแข่งกี่คน", "ใครขับรถ Toyota ปีนี้"],
        "responses": [
            "สำหรับฤดูกาล {year} นักขับหลักของทีมคือ {drivers[0]}, {drivers[1]} และ {drivers[2]} รวมถึง {drivers[3]} ที่ลงแข่งบางสนาม",
            "รายชื่อนักขับของ Gazoo Racing ในปีนี้ได้แก่ {drivers[0]}, {drivers[1]}, {drivers[2]} และ {drivers[3]}",
            "ทีมมีนักขับหลัก 3 คนคือ {drivers[0]}, {drivers[1]} และ {drivers[2]} พร้อมด้วยนักขับพาร์ทไทม์อย่าง {drivers[3]}"
        ]
    },
    "drivers_history_kalle": {
        "questions": ["ประวัติ Kalle Rovanperä", "Rovanperä มีผลงานยังไงบ้าง", "ประวัติของ Kalle", "Kalle Rovanperä เคยได้แชมป์ไหม", "ข้อมูลของ Kalle", "Rovanperä", "ผลงานของ Rovanperä", "Kalle"],
        "responses": [
            "{drivers[0]} เป็นแชมป์โลก WRC 2 สมัยติดต่อกัน (2022, 2023) และเป็นนักขับอายุน้อยที่สุดที่คว้าแชมป์โลกได้"
        ]
    },
    "drivers_history_elfyn": {
        "questions": ["ประวัติ Elfyn Evans", "Elfyn Evans มีผลงานยังไงบ้าง", "ประวัติของ Elfyn", "ข้อมูลของ Elfyn", "Elfyn Evans", "Evans", "ผลงานของ Evans", "Elfyn"],
        "responses": [
            "{drivers[1]} เป็นรองแชมป์โลกปีล่าสุดและเป็นกำลังสำคัญของทีม"
        ]
    },
    "drivers_history_taka": {
        "questions": ["ประวัติ Takamoto Katsuta", "Takamoto Katsuta มีผลงานยังไงบ้าง", "ประวัติของ Taka", "Taka Katsuta", "Taka", "Katsuta", "ผลงานของ Takamoto"],
        "responses": [
            "{drivers[2]} มีพัฒนาการที่น่าจับตามองและเคยขึ้นโพเดียมมาแล้วหลายครั้ง"
        ]
    },
    "drivers_history_ogier": {
        "questions": ["ประวัติ Sébastien Ogier", "Sébastien Ogier มีผลงานยังไงบ้าง", "ประวัติของ Ogier", "Ogier", "Sébastien", "Ogier แข่งให้ทีมนี้ด้วยเหรอ", "ผลงานของ Ogier"],
        "responses": [
            "ใช่ครับ {drivers[3]} เป็นนักขับพาร์ทไทม์ที่ช่วยทีมเก็บคะแนนในบางสนามสำคัญ"
        ]
    },
    "team_history": {
        "questions": ["ประวัติทีม", "ทีมก่อตั้งเมื่อไหร่", "ทีมเคยได้แชมป์อะไรมาบ้าง", "ประวัติของทีม", "ความเป็นมาของทีม Gazoo Racing", "เรื่องราวของทีม", "สถิติทีม"],
        "responses": [
            "ทีม Gazoo Racing ได้รับการก่อตั้งขึ้นอย่างเป็นทางการในปี 2017 และสามารถคว้าแชมป์โลกผู้ผลิตได้หลายครั้ง รวมถึงแชมป์โลกนักขับกับ Sébastien Ogier และ Kalle Rovanperä"
        ]
    },
    "drivers_standings_2025": {
        "questions": [
            "อันดับนักขับของ {team_short} ปี {year} ตอนนี้",
            "คะแนนแชมป์นักขับของ {team_short} {year}",
            "คะแนนของ {drivers[0]} ปี {year} เท่าไหร่",
            "คะแนนของ {drivers[1]} ปี {year} เท่าไหร่",
            "คะแนนของ {drivers[2]} เท่าไหร่",
            "สรุป standings นักขับ {team_short} {year}",
            "คะแนนนักขับ",
            "ใครนำในตารางคะแนนนักขับ",
            "คะแนนสะสมนักแข่ง",
            "อันดับในตารางนักขับ"
        ],
        "responses": [
            "อัปเดตถึง {asof_event} ({asof_date}) — {drivers[1]} อยู่อันดับ {standings['Elfyn Evans']['pos']} มี {standings['Elfyn Evans']['pts']} คะแนน, {drivers[0]} อยู่อันดับ {standings['Kalle Rovanperä']['pos']} มี {standings['Kalle Rovanperä']['pts']} คะแนน, {drivers[3]} อยู่อันดับ {standings['Sébastien Ogier']['pos']} มี {standings['Sébastien Ogier']['pts']} คะแนน และ {drivers[2]} อยู่อันดับ {standings['Takamoto Katsuta']['pos']} มี {standings['Takamoto Katsuta']['pts']} คะแนน",
            "อันดับนักขับของทีม ณ {asof_event}: {drivers[1]} (P{standings['Elfyn Evans']['pos']}), {drivers[0]} (P{standings['Kalle Rovanperä']['pos']}), {drivers[3]} (P{standings['Sébastien Ogier']['pos']}) และ {drivers[2]} (P{standings['Takamoto Katsuta']['pos']})."
        ]
    },
    "team_principal": {
        "questions": ["หัวหน้าทีม {team_short} คือใคร","team principal {team_short}", "ใครเป็นหัวหน้าทีม", "ใครคุมทีม", "หัวหน้าทีมคือ", "ผู้จัดการทีมคือใคร", "ใครบริหารทีม"],
        "responses": [
            "หัวหน้าทีม (Team Principal) ของ Gazoo Racing คือ {team_principal} อดีตนักขับ WRC ชื่อดัง"
        ]
    },
    "technical_director": {
        "questions": ["ผู้อำนวยการฝ่ายเทคนิค {team_short} คือใคร","technical director {team_short}", "ใครคุมทีมฝ่ายเทคนิค", "ใครดูแลเรื่องเทคนิค", "ใครคุมการพัฒนารถ", "ใครเป็นหัวหน้าวิศวกร"],
        "responses": [
            "ผู้อำนวยการฝ่ายเทคนิคของทีมคือ {technical_director} ซึ่งรับผิดชอบการพัฒนาด้านเทคนิคของรถแข่ง"
        ]
    },
    "bases": {
        "questions": ["ฐานทีม {team_short} อยู่ที่ไหน","โรงงาน {team_short} อยู่ไหน","where is {team_short} based", "ที่ตั้งทีม", "ที่ตั้งโรงงาน", "ทีมมีฐานอยู่ที่ใด", "สำนักงานใหญ่ของทีมอยู่ที่ไหน"],
        "responses": [
            "ฐานใหญ่ของทีมอยู่ที่ {base_chassis} ส่วนการพัฒนาเครื่องยนต์และเทคโนโลยีส่วนใหญ่เกิดขึ้นที่ศูนย์วิจัยใน {base_engine}"
        ]
    },
    "car_name": {
        "questions": ["รถปี {year} ของทีมชื่ออะไร","ชื่อรถ {team_short} {year}", "รถแข่งของทีมคืออะไร", "รถของทีมชื่ออะไร", "ทีมใช้รถอะไร", "รถคันไหนที่ใช้แข่ง"],
        "responses": [
            "รถแข่งของทีมในฤดูกาลนี้คือ {car_name}."
        ]
    },
    "car_details": {
        "questions": ["รายละเอียดรถ {car_name}", "รถใช้เครื่องยนต์อะไร", "รถมีจุดเด่นอะไร", "สเป็ครถเป็นยังไง", "รถคันนี้เป็นยังไง", "รถมีคุณสมบัติอะไรบ้าง", "สเป็ครถ Toyota Yaris Rally1"],
        "responses": [
            "{car_name} ใช้เครื่องยนต์เบนซิน 1.6 ลิตร เทอร์โบ ผสานกับระบบไฮบริด 100kW (134 แรงม้า) ตามกติกา FIA Rally1",
            "จุดเด่นของรถคือการออกแบบที่เน้นความสมดุลและประสิทธิภาพในการขับขี่บนพื้นผิวที่หลากหลาย"
        ]
    },
    "car_focus_2025": {
        "questions": ["รถปีนี้พัฒนาจากปีที่แล้วตรงไหนบ้าง","จุดเด่นรถปีนี้คืออะไร", "มีการปรับปรุงอะไรจากรถรุ่นก่อนบ้าง", "การพัฒนาของรถปี 2025", "รถปีใหม่มีอะไรเด่น"],
        "responses": [
            "ทีมเน้นการปรับปรุงระบบแอร์โรไดนามิก, ช่วงล่าง และการจัดการพลังงานไฮบริดเพื่อเพิ่มประสิทธิภาพในทุกสภาพถนน"
        ]
    },
    "power_unit": {
        "questions": ["เครื่องยนต์ของ {team_short} {year} คืออะไร","engine {team_short} {year}", "รถใช้เครื่องยนต์แบบไหน", "ระบบไฮบริดเป็นยังไง", "เครื่องยนต์ของรถ", "รถใช้พลังงานอะไร", "เครื่องยนต์ไฮบริด"],
        "responses": [
            "รถแข่งใช้เครื่องยนต์แบบ {power_unit} ซึ่งเป็นการผสานพลังงานแบบดั้งเดิมเข้ากับเทคโนโลยีไฮบริด"
        ]
    },
    "constructors_standings_asof": {
        "questions": ["คะแนนทีม {team_short} {year} ตอนนี้","ทีม Gazoo Racing มีคะแนน Manufacturers เท่าไหร่", "อันดับทีมล่าสุด", "คะแนนทีมผู้ผลิต", "ใครนำในตารางทีม", "อันดับผู้ผลิตตอนนี้"],
        "responses": [
            "{constructors_points_asof} ในขณะที่ Hyundai Shell Mobis อยู่อันดับสอง"
        ]
    },
    "results_live_disclaimer": {
        "questions": ["ผลล่าสุดวันนี้","ทีมชนะไหมวันนี้","คะแนนสด", "ใครชนะ", "อัปเดตผลสด", "ผลคะแนนตอนนี้", "อัปเดตการแข่ง", "วันนี้ใครได้แชมป์", "ผลการแข่งขันล่าสุด", "สรุปผลการแข่งขัน"],
        "responses": [
            "ผมไม่สามารถบอกผลแบบเรียลไทม์ได้ ข้อมูลล่าสุดของผมอัปเดตถึง {asof_date} หลังจบ {asof_event}"
        ]
    },
    "team_rivals": {
        "questions": ["คู่แข่งหลักของ Gazoo Racing คือใคร", "ทีมไหนแข่งกับ {team_short} ปี {year}", "ใครเป็นคู่แข่งที่น่ากลัวที่สุด"],
        "responses": [
            "คู่แข่งหลักในฤดูกาล {year} ได้แก่ Hyundai Shell Mobis และ M-Sport Ford 🏎️",
            "ทีมที่เบียดแย่งแชมป์กับ {team_short} มากที่สุดคือ Hyundai"
        ]
    },
    "out_of_scope": {
        "questions": [
            "สภาพอากาศวันนี้", "ราคา BTC วันนี้", "ผลการแข่ง le mans 24 hours", "ราคาทอง", "Ferrari 488 pista ใช้เครื่องกี่สูบ", "ข่าวการเมืองวันนี้",
            "ทีม hyundai", "ทีม hyundai ปี 2025", "ทีม bmw", "ทีม m-sport",
            "ไก่ทอด", "ปลา", "ผลบอล", "วันนี้กินอะไรดี", "มีอะไร", "มีอะไรให้ช่วยอีกไหม", "อะไรก็ได้", "ข่าว", "สวัสดีปีใหม่"
        ],
        "responses": [
            "ขออภัยครับ บอทนี้ตอบเฉพาะเรื่องที่เกี่ยวข้องกับทีม WRC Toyota Gazoo Racing ในปี {year} เท่านั้นครับ/ค่ะ"
        ]
    }
}


In [131]:
# ---------- augmentation + dataset builder ----------
EMOJIS = ["🏎️","✨","🔥","💬","✅","❓","📊","🏁"]
PUNCTS = ["","!","!!","!!!","?","??","..."]
TEAM_SYNS = [
    "WRC Gazoo Racing","WRC Toyota","Gazoo Racing","GR Team",
    "Toyota Gazoo Racing","ทีม Gazoo Racing","ทีมโตโยต้า"
]
YEAR_SYNS = [FACTS["year"], f"ฤดูกาล {FACTS['year']}", f"season {FACTS['year']}", f"{FACTS['year']} season"]

def augment(text: str) -> str:
    t = text.strip()
    r = random.random()
    if r < 0.1: t = t.upper()
    elif r < 0.2: t = t.capitalize()
    if random.random() < 0.35: t += " " + random.choice(EMOJIS)
    if random.random() < 0.35: t += random.choice(PUNCTS)
    if random.random() < 0.1 and len(t.split())==1 and len(t)>3:
        t = t[:-1] + t[-1]*random.randint(2,4)
    return t

def expand_question(q):
    return q.replace("{team_short}", random.choice(TEAM_SYNS)).replace("{year}", random.choice(YEAR_SYNS))

def build_dataset(target_rows=9000, seed=42):
    random.seed(seed)
    rows = []
    intents = list(INTENT_TEMPLATES.keys())
    per_intent = max(200, target_rows // len(intents))
    for intent, pack in INTENT_TEMPLATES.items():
        qs = pack["questions"]
        rs = [fill_response(r) for r in pack["responses"]]
        for _ in range(per_intent):
            q = augment(expand_question(random.choice(qs)))
            r = augment(random.choice(rs))
            rows.append((q, intent, r))
    random.shuffle(rows)
    return pd.DataFrame(rows, columns=["question","intent","response"])

In [132]:
# --- สร้างและบันทึก dataset ---
df = build_dataset(target_rows=9000)
df.to_excel("qa_dataset.xlsx", index=False)
print(" สร้าง qa_dataset.xlsx จำนวนแถว:", len(df))

 สร้าง qa_dataset.xlsx จำนวนแถว: 8993


In [133]:
# =========================================================
# 2. Train, Evaluate, Save Model
# =========================================================

# Load (จากไฟล์ที่เพิ่งสร้าง)
df = pd.read_excel("qa_dataset.xlsx")

X, y = df["question"], df["intent"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

responses_dict = df.groupby("intent")["response"].apply(list).to_dict()

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.9817
                             precision    recall  f1-score   support

                      bases       0.93      1.00      0.96        78
               capabilities       1.00      1.00      1.00        78
                car_details       1.00      1.00      1.00        78
             car_focus_2025       1.00      1.00      1.00        78
                   car_name       1.00      1.00      1.00        78
constructors_standings_asof       1.00      1.00      1.00        78
               drivers_2025       1.00      0.92      0.96        79
      drivers_history_elfyn       1.00      1.00      1.00        79
      drivers_history_kalle       1.00      1.00      1.00        78
      drivers_history_ogier       1.00      0.99      0.99        78
       drivers_history_taka       1.00      0.97      0.99        79
     drivers_standings_2025       0.93      1.00      0.96        78
                    goodbye       1.00      0.99      0.99        78
                

In [134]:
# Save model
joblib.dump(model, "chatbot_model.pkl")
print("Model saved as chatbot_model.pkl")

Model saved as chatbot_model.pkl


In [135]:
import random
def chatbot_response(user_input: str):
    intent = model.predict([user_input])[0]
    fallback = f"ขอโทษครับ/ค่ะ บอทนี้ตอบเฉพาะ {FACTS['team_short']} ฤดูกาล {FACTS['year']} เท่านั้น"
    return random.choice(responses_dict.get(intent, [fallback]))

print(chatbot_response("นักขับของ GR Team ปี 2025 มีใครบ้าง"))
print(chatbot_response("รถปี 2025 ของทีมชื่ออะไร"))
print(chatbot_response("หัวหน้าทีมคือใคร"))

สำหรับฤดูกาล 2025 นักขับหลักของทีมคือ kalle rovanperä, elfyn evans และ takamoto katsuta รวมถึง sébastien ogier ที่ลงแข่งบางสนาม
รถแข่งของทีมในฤดูกาลนี้คือ Toyota GR Yaris Rally1 Hybrid. ✨
หัวหน้าทีม (Team Principal) ของ Gazoo Racing คือ Jari-Matti Latvala อดีตนักขับ WRC ชื่อดัง


In [136]:
%%writefile app.py
import os
import streamlit as st
import pandas as pd
import random
import joblib

# ปิด usage stats
os.environ["STREAMLIT_BROWSER_GATHERUSAGESTATS"] = "false"

MODEL_PATH = "chatbot_model.pkl"
DATASET_PATH = "qa_dataset.xlsx"

# โหลดโมเดล/ชุดคำตอบ
try:
    model = joblib.load(MODEL_PATH)
    df = pd.read_excel(DATASET_PATH)
    intent_to_response = df.groupby("intent")["response"].apply(list).to_dict()
    is_ready = True
except FileNotFoundError:
    st.error("ไม่พบไฟล์ 'chatbot_model.pkl' หรือ 'qa_dataset.xlsx' โปรดตรวจสอบว่าคุณได้รันโค้ดส่วนก่อนหน้าแล้ว")
    is_ready = False

st.set_page_config(page_title="WRC Toyota Gazoo Racing 2025 Chatbot", page_icon="🏁")
st.title("WRC Toyota Gazoo Racing 2025 Chatbot")
st.caption("สอบถามข้อมูลเกี่ยวกับทีม WRC Toyota Gazoo Racing ฤดูกาล 2025 ได้ที่นี่")

# session state
if "history" not in st.session_state:
    st.session_state.history = []

# input
user_input = st.text_input("คุณ: ", "", placeholder="ถามเกี่ยวกับทีม WRC Toyota Gazoo Racing 2025...")

col1, col2 = st.columns([1,1])
with col1:
    send = st.button("ส่ง", type="primary", disabled=not is_ready)
with col2:
    clear = st.button("ล้างประวัติ")

if clear:
    st.session_state.history = []

if send and user_input.strip() and is_ready:
    predicted_intent = model.predict([user_input])[0]
    fallback = "ขอโทษครับ/ค่ะ บอทนี้ตอบเฉพาะเรื่องที่เกี่ยวข้องกับทีม WRC Toyota Gazoo Racing 2025 เท่านั้น"
    bot_reply = random.choice(
        intent_to_response.get(predicted_intent, [fallback])
    )
    st.session_state.history.append({"user": user_input, "bot": bot_reply})

# history
for chat in st.session_state.history:
    st.markdown(f"**คุณ:** {chat['user']}")
    st.markdown(f"**🤖 บอท:** {chat['bot']}")

Overwriting app.py


In [137]:
!ngrok authtoken 2spnfIOdpl97fYtMczqo8uE2oDf_3Rq1fBtjHEU798kjtpzet

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [139]:
!streamlit run app.py --server.port 8501 &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.187.139.105:8501

  Stopping...


In [138]:
# เปิดทางเข้า ngrok
from pyngrok import ngrok
ngrok.kill()
public_url = ngrok.connect(8501)
print("🌍 Open your app here:", public_url)

🌍 Open your app here: NgrokTunnel: "https://523c80adebfd.ngrok-free.app" -> "http://localhost:8501"
